## 5.1 利用nn.Module构建线性回归模型

nn.Module里有一个子模块叫做nn.Linear,顾名思义，它进行的是wb+b的线性计算。


### 5.1.1 构建模型
还记得第四章我们是如何建立线性模型的吗，我们写了一个函数：

In [1]:
def model(x, w, b):
    return w*x + b

现在我们用nn.Linear去代替这个自己撰写函数

In [2]:
import torch.nn as nn

linear_model = nn.Linear(1,1)

nn.Linear(1,1)构建了一个一元线性回归模型结构实例，第一个参数代表输入的特征数，第二个参数代表输出的个数，因为是一元回归，所有输入是1，输出也是1。同时，已经在该实例内部初始化了模型需要的参数w,b，无需自己创建于初始化，可以如下查看参数：

In [5]:
linear_model.weight

Parameter containing:
tensor([[-0.6442]], requires_grad=True)

In [6]:
linear_model.bias

Parameter containing:
tensor([0.1683], requires_grad=True)

In [13]:
linear_model.parameters()
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.6442]], requires_grad=True), Parameter containing:
 tensor([0.1683], requires_grad=True)]

### 5.1.2 单个样本前向计算
构建好了模型，就可以拿去做前向计算了：

In [4]:
import torch

x = torch.tensor([0.5])

# 前向计算
y = linear_model(x)
print(y)


tensor([-0.1538], grad_fn=<ThAddBackward>)


以上，调用模型实例linear_model输入参数x,其实是调用了nn.Linear类中的方法forward(),模型的前向计算过程会全部写在forward方法中。

###  5.1.3 批量样本前向计算
nn中的所有module都是可以同时输入批量多元的input，并输出批量多元的output的。比如你想一次性处理10个样本，你只需将数据处理成B*n_in大小的tensor一次性喂给模型，它会输出B*n_out的大小的tensor,其中B是批量数据的数量，n_in是输入特征的数目，n_out是输出特征的数目。比如：

In [7]:
# 生成10个样本，输入特征数为1
x = torch.ones(10,1)
print(x)

y = linear_model(x)
print(y)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])
tensor([[-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758],
        [-0.4758]], grad_fn=<ThAddmmBackward>)


注意B个数据的大小应为B*n_in，是两维的tensor, 还记得第四章的老例子吗，x和y的tensor大小是一维的,需要做如下转换：
.unsqueeze(1)指axis=1处额外增加一个维度

In [8]:
x = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
y = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
x = torch.tensor(x).unsqueeze(1)
y = torch.tensor(y).unsqueeze(1)

print(x)
print(x.shape)

tensor([[ 0.5000],
        [14.0000],
        [15.0000],
        [28.0000],
        [11.0000],
        [ 8.0000],
        [ 3.0000],
        [-4.0000],
        [ 6.0000],
        [13.0000],
        [21.0000]])
torch.Size([11, 1])


### 5.1.4 完整训练过程

使用Pytorch提供的module进行一元线性回归模型的构建于训练，完整代码如下：

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1.准备数据
x = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
y = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
x = torch.tensor(x).unsqueeze(1)
y = torch.tensor(y).unsqueeze(1)

n_sample = x.shape[0]  # 总样本数
n_val = int(0.2 * n_sample)

shuffled_indices = torch.randperm(n_sample) # 获取随机正整数

train_indices = shuffled_indices[: -n_val] # 训练集的索引
val_indices = shuffled_indices[-n_val:] # 验证集的索引

train_x = 0.1 * x[train_indices]  # 别忘了乘以0.1缩小x规模
train_y = y[train_indices]

val_x = 0.1 * x[val_indices]
val_y = y[val_indices]

# 2.构建模型
linear_model = nn.Linear(1, 1)

# 3.构建优化器
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)  # 注意直接通过模型实例获取参数，无需自己创建参数

# 4.训练
def training_loop(n_epochs, optimizer, model, loss_fn, train_x, val_x, train_y, val_y):
    for epoch in range(1, n_epochs+1):
        # 训练集的前向
        train_y_p = model(train_x)
        train_loss = loss_fn(train_y_p, train_y)
        
        # 验证集的前向
        val_y_p = model(val_x)
        val_loss = loss_fn(val_y_p, val_y)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch % 100 == 0:
            print('Epoch {}, Train Loss {}, Val Loss {}'.format(epoch, float(train_loss), float(val_loss)))

training_loop(
    n_epochs = 1000,
    optimizer = optimizer,
    model = linear_model,   # 注意这里
    loss_fn = nn.MSELoss(),  # 注意这里，直接调用pytorch提供的损失类，其实它也是一个子modual
    train_x = train_x,
    val_x = val_x,
    train_y = train_y,
    val_y = val_y
)
                                                                                                                                                                                       

Epoch 100, Train Loss 32.69783020019531, Val Loss 59.25724792480469
Epoch 200, Train Loss 15.095704078674316, Val Loss 35.80534362792969
Epoch 300, Train Loss 10.374916076660156, Val Loss 25.756357192993164
Epoch 400, Train Loss 9.098362922668457, Val Loss 21.192996978759766
Epoch 500, Train Loss 8.753169059753418, Val Loss 18.99950408935547
Epoch 600, Train Loss 8.659825325012207, Val Loss 17.907453536987305
Epoch 700, Train Loss 8.634584426879883, Val Loss 17.35272216796875
Epoch 800, Train Loss 8.62775993347168, Val Loss 17.06779670715332
Epoch 900, Train Loss 8.62590503692627, Val Loss 16.920541763305664
Epoch 1000, Train Loss 8.625411033630371, Val Loss 16.844257354736328


上面的过程，和4.4中是一样的结果，区别就在于，我们用pytorch提供的module代替了手工写的模型（nn.Linear）与损失函数（nn.MSELoss()）,也无需自己创建于初始化参数，在创建模型实例的时候会自动创建，只需module.parameters就能获取参数。

module.parameters是开启了requries_grad=True的，因此当loss.backwards的时候，会对parameters计算梯度，并在optimizer.step时累加更新梯度。

到目前为止，我们已经学会了如何用Pytorch去创建模型，构建优化器与损失函数，并进行迭代训练。虽然只是用了最简单的线性回归，写来写去都是这么二十几行代码，但是这确是在授人以渔。接下去你可以将线性模型替换成其他复杂的模型,可以将SGD换成其他更优秀的优化器，将MSEloss换成其他损失函数，去自定义创造自己的项目，钓自己的鱼。

至于pytorch提供了什么submodule, optimizer, loss function,以及其他各种功能，可以直接去官网文档中一探究竟。